<div id = "monlogo"><center> <img src="https://datascientest.fr/train/assets/logo_datascientest.png", style="height:150px"></center></div>

<hr style="border-width:2px;border-color:#75DFC1"><br>
<center> <h1>Challenge Sorbonne</h1> </center> <br>
<center> <h3>Exemple Modélisation</h3> </center> <br>
<hr style="border-width:2px;border-color:#75DFC1"> 

### Modélisation Machine Learning

>  L'objectif est de créer un modèle permettant d'**identifier pour chaque course de la saison 2021, le vainqueur ainsi que le podium**. Il faut donc créer un modèle de classification binaire, ainsi qu'un modèle de classification multiclasse. Pour rappel, chacun de ces 2 modèles servira pour la moitié de la note de modélisation.
>
> Il est nécessaire de garder ces éléments en tête :
> - **Données futures** : Les Grand Prix ont lieu les uns après les autres, et vous avez dans les différentes tables accès à des données qui n'existent pas encore au moment du départ d'un Grand Prix. Créer un modèle reposant sur des informations futures serait un non-sens mathématiques et une grosse erreur de méthodologie.
> - **Preprocessing** : Il y a bon nombre de données manquantes, incomplètes ou inutiles dans le jeu de données, faites le tri. 
> - **Feature Engineering** : Vous avez un grand nombre de tables, certaines contiennent des features directement exploitables pour le machine learning, d'autres sont intéressantes mais nécessitent des modifications afin d'être utilisées à bon escient. Vous avez également la possibilité d'augmenter vos données à l'aide de librairies comme [FastF1](https://theoehrly.github.io/Fast-F1/), ou en scrapant sur le web.
> - **Métriques et interprétabilités** : Vos modèles seront évaluées selon la métrique bien connue du [f1-score](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html), prenez cela en compte lors de l'optimisation de vos modèles.

### Exemple

> Vous trouverez ci-dessous un exemple de modèle naïf de classification binaire réalisé à partir des tables `results.csv`, `races.csv`, `circuits.csv` et `constructor_standings.csv`, se basant uniquement sur **la position de départ de chaque pilote sur la grille**, **le pays du circuit**, **le classement et le nombre de points constructeur au moment du départ**.
>
> Attention, ce travail de modélisation a uniquement vocation à vous donner une piste pour démarrer, celui-ci n'est pas parfaitement optimisé et ne constitue pas un bon exemple à suivre.

In [123]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

In [124]:
#chargemnt des données
drivers = pd.read_csv(r"data\driver_standings.csv", sep=",") #on peut prendre le raceID, driverID, Points, Position, wins
driversINFO = pd.read_csv(r"data\drivers.csv",sep=",") #on prend la date de naisse dob[:4]
races = pd.read_csv(r"data\races.csv",sep=",") #raceID, year, circuitID, date, time
results = pd.read_csv(r"data\results.csv", sep=",") #driverID, raceID, positionORder, grid

results = results[['raceId','driverId','grid','positionOrder']] 
driversINFO = driversINFO[['driverId','dob']]
drivers = drivers[['raceId','driverId','points','wins']]
races = races[['raceId','year','circuitId','date','time']]

In [125]:
#transformation pour avoir la date de naissance des conducteurs
driversINFO.loc[:,'dob'] = pd.to_datetime(driversINFO['dob']).dt.year

In [126]:
#transformation de la date des courses pour avoir le mois 
races.loc[:,'date'] = pd.to_datetime(races['date']).dt.month
races = races.rename(columns={'date' : 'month'})

In [127]:
#retirer une erreur dans les dates et garder uniquement l'heure de la course
races["time"] = races['time'].replace({'\\N' : '12:00:00'})
races['hour'] = pd.to_datetime(races['time']).dt.hour
races = races.drop(columns = ["time"])

C:\Users\luang\AppData\Local\Temp\ipykernel_8988\4191630194.py:3: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  races['hour'] = pd.to_datetime(races['time']).dt.hour


In [131]:
results.head()

,raceId,driverId,grid,positionOrder
0,18,1,1,1.0
1,18,2,5,2.0
2,18,3,7,3.0
3,18,4,11,4.0
4,18,5,3,5.0


In [130]:
driversINFO.head()

,driverId,dob
0,1,1985
1,2,1977
2,3,1985
3,4,1981
4,5,1981


In [129]:
drivers.head()

,raceId,driverId,points,wins
0,18,1,10.0,1
1,18,2,8.0,0
2,18,3,6.0,0
3,18,4,5.0,0
4,18,5,4.0,0


In [128]:
races.head()

,raceId,year,circuitId,month,hour
0,1,2009,1,3,6
1,2,2009,2,4,9
2,3,2009,17,4,7
3,4,2009,3,4,12
4,5,2009,4,5,12


In [132]:
# on joint resultat et races sur la base de raceID
df1 = pd.merge(results[['raceId','driverId','grid','positionOrder']], races[['raceId','year','circuitId','month','hour']], on = 'raceId')


In [133]:
# on joint le précédent avec drivers avec drivers info sur driverID
df2 = pd.merge(drivers, driversINFO, on = 'driverId')

In [134]:
df3 = pd.merge(df1,df2, on = ['raceId', 'driverId'])
df3

,raceId,driverId,grid,positionOrder,year,circuitId,month,hour,points,wins,dob
0,18,1,1,1.0,2008,1,3,4,10.0,1,1985
1,18,2,5,2.0,2008,1,3,4,8.0,0,1977
2,18,3,7,3.0,2008,1,3,4,6.0,0,1985
3,18,4,11,4.0,2008,1,3,4,5.0,0,1981
4,18,5,3,5.0,2008,1,3,4,4.0,0,1981
...,...,...,...,...,...,...,...,...,...,...,...
25366,1096,854,12,NaN,2022,24,11,13,12.0,0,1999
25367,1096,825,16,NaN,2022,24,11,13,25.0,0,1992
25368,1096,1,5,NaN,2022,24,11,13,240.0,0,1985
25369,1096,849,20,NaN,2022,24,11,13,2.0,0,1995


In [135]:
#on utilise positionOrder pour faire le winner
df3['winner'] = df3['positionOrder'].apply(lambda x: 1 if x==1 else 0)
df3["AgeAtRace"] = df3['year'] - df3['dob']
df4 = df3.drop(['positionOrder','circuitId', 'dob'], axis = 1).set_index(['raceId', 'driverId'])

In [136]:
df4

grid  year month  hour  points  wins  winner AgeAtRace
raceId driverId                                                        
18     1            1  2008     3     4    10.0     1       1        23
       2            5  2008     3     4     8.0     0       0        31
       3            7  2008     3     4     6.0     0       0        23
       4           11  2008     3     4     5.0     0       0        27
       5            3  2008     3     4     4.0     0       0        27
...               ...   ...   ...   ...     ...   ...     ...       ...
1096   854         12  2022    11    13    12.0     0       0        23
       825         16  2022    11    13    25.0     0       0        30
       1            5  2022    11    13   240.0     0       0        37
       849         20  2022    11    13     2.0     0       0        27
       4           10  2022    11    13    81.0     0       0        41

[25371 rows x 8 columns]

In [137]:
#on retire qui a gagné la course à chaque course
df4["wins"] = df4["wins"] - df4["winner"]

In [138]:
df4

grid  year month  hour  points  wins  winner AgeAtRace
raceId driverId                                                        
18     1            1  2008     3     4    10.0     0       1        23
       2            5  2008     3     4     8.0     0       0        31
       3            7  2008     3     4     6.0     0       0        23
       4           11  2008     3     4     5.0     0       0        27
       5            3  2008     3     4     4.0     0       0        27
...               ...   ...   ...   ...     ...   ...     ...       ...
1096   854         12  2022    11    13    12.0     0       0        23
       825         16  2022    11    13    25.0     0       0        30
       1            5  2022    11    13   240.0     0       0        37
       849         20  2022    11    13     2.0     0       0        27
       4           10  2022    11    13    81.0     0       0        41

[25371 rows x 8 columns]

In [ ]:
#'grid','year','month','hour','points','wins','AgeAtRace'

In [139]:
#On effectue notre train_test_split selon les dates puis on sépare X et y
df_train = df4[df4['year']<=2020]
df_test = df4[df4['year']==2021]

X_train = df_train[['grid','year','month','hour','points','wins','AgeAtRace']]
X_test = df_test[['grid','year','month','hour','points','wins','AgeAtRace']]

y_train = df_train[['winner']]
#y_test ne contient pas d'informations, puisque c'est ce qu'il faut prédire et soumettre sur la plateforme :)
y_test = df_test[['winner']]
y_pred = df_test[['winner']]

In [140]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Définition du modèle
random_forest = RandomForestClassifier(random_state = 42)

# Définition de la grille de paramètres et du GridSearchCV
param_grid = {
    'n_estimators': [25, 50, 75, 100,150, 200],
    'max_depth': [5, 10, 20, 30, 40, 50]
}

grid_search = GridSearchCV(random_forest, param_grid, scoring='recall')

# Entraînement du modèle
grid_search.fit(X_train, y_train.values.ravel())
print(grid_search.best_params_)

{'max_depth': 20, 'n_estimators': 75}


In [141]:
print(grid_search.score(X_train,y_train))

0.9990366088631984


In [143]:
probabilite = grid_search.predict_proba(X_test)
probabilite = pd.DataFrame(probabilite)
probabilite = probabilite.drop(columns=(0))
probabilite = np.array(probabilite)
y_pred.loc[:,'PREpositionOrder'] = probabilite

C:\Users\luang\AppData\Local\Temp\ipykernel_8988\3762631047.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'PREpositionOrder'] = probabilite


In [144]:
y_pred

winner  PREpositionOrder
raceId driverId                          
1052   1              0          0.480000
       830            0          0.360000
       822            0          0.213333
       846            0          0.093333
       815            0          0.360000
...                 ...               ...
1073   849            0          0.000000
       841            0          0.000000
       847            0          0.013333
       8              0          0.000000
       853            0          0.000000

[440 rows x 2 columns]

In [145]:
Taille = y_pred.index
raceID = Taille.get_level_values('raceId').unique()
PodiumOr0 = []
Top1 = []
for RACE in raceID:
    driverID = y_pred["PREpositionOrder"][RACE].index.get_level_values('driverId')
    array = np.array(y_pred["PREpositionOrder"][RACE])
    ranks = array.argsort()
    ranks = 20 - ranks.argsort()
    count = 0
    for DRIVER in driverID:
        rang = ranks[count]
        if rang <= 3:
            PodiumOr0.append(ranks[count])
            if rang == 1:
                Top1.append(1)
            else:
                Top1.append(0)
        else:
            PodiumOr0.append(0)
            Top1.append(0)
        count += 1 
    count = 0

In [146]:
print(len(PodiumOr0),len(Top1))

440 440


In [147]:
y_pred.loc[:,'winner'] = Top1
y_pred.loc[:,'positionOrder'] = PodiumOr0
y_pred = y_pred.drop(columns=("PREpositionOrder"))

C:\Users\luang\AppData\Local\Temp\ipykernel_8988\1890261867.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred.loc[:,'positionOrder'] = PodiumOr0


In [148]:
y_pred

winner  positionOrder
raceId driverId                       
1052   1              1              1
       830            0              2
       822            0              0
       846            0              0
       815            0              3
...                 ...            ...
1073   849            0              0
       841            0              0
       847            0              0
       8              0              0
       853            0              0

[440 rows x 2 columns]

In [149]:
y_pred.to_csv('model2OnlyDriver.csv')

score : 62.61